In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../..'))
	print(os.getcwd())
except:
	pass


 # 1. FCI用のnamespace作成

In [25]:
%%bash
kubectl create namespace mssql-fci


namespace/mssql-fci created


 # 2. FCI用のsecret作成

In [26]:
%%bash
kubectl create secret generic mssql --from-literal=SA_PASSWORD=$PASSWORD -n mssql-fci


secret/mssql created


 # 3. PVC作成

In [27]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/FCI/pvc.yaml


kind: StorageClass
apiVersion: storage.k8s.io/v1beta1
metadata:
  name: azure-disk
provisioner: kubernetes.io/azure-disk
parameters:
  storageaccounttype: Standard_LRS
  kind: Managed
---
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: mssql-data
  annotations:
    volume.beta.kubernetes.io/storage-class: azure-disk
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 8Gi

In [29]:
%%bash
kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/FCI/pvc.yaml -n mssql-fci


storageclass.storage.k8s.io/azure-disk created
persistentvolumeclaim/mssql-data created


 # 4. Pod作成

In [30]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/FCI/sql19deployment.yaml


apiVersion: apps/v1beta1
kind: Deployment
metadata:
  name: mssql-deployment
spec:
  replicas: 1
  template:
    metadata:
      labels:
        app: mssql
    spec:
      terminationGracePeriodSeconds: 10
      containers:
      - name: mssql
        image: mcr.microsoft.com/mssql/server:2019-CU1
        ports:
        - containerPort: 1433
        env:
        - name: MSSQL_PID
          value: "Developer"
        - name: ACCEPT_EULA
          value: "Y"
        - name: MSSQL_SA_PASSWORD
          valueFrom:
            secretKeyRef:
              name: mssql
              key: SA_PASSWORD 
        volumeMounts:
        - name: mssqldb
          mountPath: /var/opt/mssql
      volumes:
      - name: mssqldb
        persistentVolumeClaim:
          claimName: mssql-data
---
apiVersion: v1
kind: Service
metadata:
  name: mssql-deployment
spec:
  selector:
    app: mssql
  ports:
    - protocol: TCP
      port: 1433
      targetPort: 1433
  type: LoadBalancer

In [31]:
%%bash
kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/FCI/sql19deployment.yaml -n mssql-fci


deployment.apps/mssql-deployment created
service/mssql-deployment created
